# Find Duplicate Transactions
This notebook provides a test bed for finding the most efficient code to ingest a list of transactions and find potential duplicates.   

It's necessary to periodically perform this task when adding downloaded mint transactions to an existing set of previously exported transactions, because there does not seem to be a way to exclude "pending" transactions from the mint export.   These pending transactions can morph a bit before they settle and can get imported twice in certain circumstances.

In [1]:
# Define the number of days to look for duplicates
DAYS = 3

# Set a dollar threshold where it's not worth the work
TOO_SMALL = 10.00

# If desired set a year to focus on
YEAR = 2024

# Define some categories that are ignored in our analyses
IGNORED_CATEGORIES = ['Credit Card Payment', 'Transfer']

# I know that my data has lots of duplicate 529 contributions
IGNORED_CATEGORIES.append('529 Contribution')
IGNORED_CATEGORIES.append('Kids Spending')

In [2]:
import pandas as pd
import read_mint_transaction_data as rmtd

# Import shared configuration file
import expenses_config as ec

# Read in the categories we'll ignore in our analysis
ignored_groups = pd.read_csv(ec.PATH_TO_GROUPS_TO_EXCLUDE)

# Read in the transaction data on disk and ensure its sorted by date
df = rmtd.read_mint_transaction_csv(ec.PATH_TO_YOUR_TRANSACTIONS, index_on_date=False)
if YEAR > 2010:
    df = df[df['Date'].dt.year == 2023]

# Ensure dates are sorted from top to bottom
df = df.sort_values('Date', ascending=False)
len(df)

2774

In [3]:
# Break transactions into groups with the same Amount, Account and Original Description
filtered_groups = df.groupby(['Original Description', 'Account Name', 'Amount'])
# for name, group in filtered_groups:
#     if len(group) > 1:
#         print(group)



Define a function that checks if a group has multiple transactions within a 3 day window

In [4]:
def has_duplicate_within_days(group, days=DAYS):
    # We'll assume that the group remains sorted
    # group = group.sort_values('Date')

    # Iterate over the group and check if adjacent dates are within the specified range
    for i in range(len(group) - 1):
        if (group['Date'].iloc[i] - group['Date'].iloc[i + 1]) <= pd.Timedelta(days=days):
            return True
    return False

Define a function that will eliminate groups from analysis based on certain criteria

In [5]:
def group_is_excluded(group):
    # Ignore if multiple transaction types..probably a refund
    if len(group) == 2 and len(group['Transaction Type'].unique()) == 2:
        return True

    # Ignore if specifically called out as a non duplicate
    if 'Not-Duplicate' in group.Labels.unique():
        return True

    return False

Define a function for breaking up groups with more than 3 transactions into smaller possible subgroups

In [6]:
def process_large_group(group):
    '''
        Break up a group of > 3 transactions into possible subgroups with
        one or more transactions within a window of DAYS

        Note that this requires the group to be sorted by date decending
    '''
    # print(f'Breaking up group for: ${name}')
    # print(group)
    group_number = 0
    group_numbers = [0]
    previous_date = group['Date'].iloc[0]  # Start with the first date

    # Iterate over the DataFrame, starting from the second row
    for date in group['Date'].iloc[1:]:
        if (previous_date - date).days > DAYS:
            group_number += 1
        group_numbers.append(group_number)
        previous_date = date

    # Add the sub group numbers into the group's dataframe and group again
    group['Group'] = group_numbers
    sub_groups = group.groupby('Group')
    for sub_name, sub_group in sub_groups:
        size = len(sub_group)
        if size == 1:
            continue
        if group_is_excluded(sub_group):
            continue
        if size == 2:
            print(sub_group)
        else:
            print('This is a big group')
            print(sub_group)

In [7]:
# Check groups to see if multiple elements are within +/- 3 Days of each other
if (YEAR < 2010):
    print('Reviewing all transactions:')
else:
    print(f'Reviewing transactions from the year {YEAR}:')
print('Will print out any possible duplicates where there are multiple transactions')
print('that have the same value for "Original Description", "Account Name", and "Amount"')
print(f'within a window of {DAYS} days\n')
if TOO_SMALL > 0:
    print(f'Transactions less than {TOO_SMALL:.2f} are ignored\n')


for name, group in filtered_groups:
    # Skip categories that we ingnore in analysis
    if len(group.Category.unique()) == 1:
        if group.Category.unique()[0] in IGNORED_CATEGORIES:
            continue

    # skip small dollar amounts -- set TOO_SMALL in first cell
    if group.Amount.unique()[0] < TOO_SMALL:
        continue

    # Ignore Rents from Apt 2F which are often the same thing twice
    if group.Labels.unique()[0] == 'Label-Rent Unit-2F' or group.Labels.unique()[0] == 'Label-Utilities Unit-2F':
        continue
    
    if len(group) > 2:
        process_large_group(group)
    elif len(group) == 2:
        # print('Evaluating group of two')
        # print(group)
        if not group_is_excluded(group) and has_duplicate_within_days(group):
            print('Possible Duplicate')
            print(group)
        # else:
        #     print('Too far apart')



Reviewing transactions from the year 2024:
Will print out any possible duplicates where there are multiple transactions
that have the same value for "Original Description", "Account Name", and "Amount"
within a window of 3 days

Transactions less than 10.00 are ignored

